In [1]:
from google.colab import files
uploaded= files.upload()

Saving winequality-red.csv to winequality-red.csv


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.linear_model import HuberRegressor
from sklearn import svm
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

# Q4
data = pd.read_csv("winequality-red.csv", sep=";")
print(data.head())
print(data.info())
print(data.describe())

data_process = data.iloc[:, :11]
data_process["const"] = 1

train_data = data_process[:1400]
train_label = data["quality"][:1400]
test_data = data_process[1400:]
test_label = data["quality"][1400:]

print(train_data.head())
print(train_label.head())
print(test_data.head())
print(test_label.head())

def calc_metric(pred_label, label):
    print(
        "Mean Absolute Error on the Test Set is : ",
        mean_absolute_error(label, pred_label) * 100,
    )


# 1 - Least-Square Loss
W = np.linalg.lstsq(train_data, train_label, rcond=None)
pred_label = test_data @ W[0]
calc_metric(pred_label, test_label)

# 2 - Huber Loss
def loss_function_huber(t):
    result = []
    for val in t:
        if abs(val) <= 0.5:
            result.append(val**2)
        else:
            result.append(abs(val) - 0.25)
    return np.mean(result)

def objective_function_huber(beta, X, Y):
    error = loss_function_huber(X @ beta - Y)
    return error

beta_init = np.array([1] * train_data.shape[1])
result_huber = minimize(
    objective_function_huber,
    beta_init,
    args=(train_data, train_label),
    method="BFGS",
    options={"maxiter": 500},
)
W_huber = result_huber.x
pred_label_huber = test_data.to_numpy() @ W_huber
calc_metric(test_label, pred_label_huber)


# 3 - Hinge Loss
def loss_function(t):
    t = [max(0, np.abs(t) - 0.5) for t in t]
    return np.mean(t)


def objective_function_hinge(beta, X, Y):
    error = loss_function(X @ beta - Y)
    return error


beta_init = np.array([1] * train_data.shape[1])
result = minimize(
    objective_function_hinge,
    beta_init,
    args=(train_data, train_label),
    method="BFGS",
    options={"maxiter": 500},
)
W = result.x
pred_label = test_data.to_numpy() @ W
calc_metric(test_label, pred_label)

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [4]:
from google.colab import files
uploaded= files.upload()

Saving ionosphere.data to ionosphere.data


In [9]:
# Q5
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.linear_model import HuberRegressor
from sklearn import svm
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

data = pd.read_csv(
    "ionosphere.data", names=[i for i in range(34)] + ["label"], header=None,
)

print(data.head())
print(data.describe())
print(data.info())

data_process = data.iloc[:, :34]
data_process["const"] = 1
data["label"] = data["label"].apply(lambda x: 1 if x == "g" else -1)

train_data = data_process[:300]
train_label = data["label"][:300]
test_data = data_process[300:]
test_label = data["label"][300:]

def calc_accuracy_metric(pred_label, test_label):
    print(
        "Accuracy on the Test Set is : ", accuracy_score(pred_label, test_label) * 100
    )

# 1 - Least Squares Loss
def loss_function(t, y):
    temp = t.T @ y
    return (temp - 1) ** 2

def objective_function(beta, X, Y):
    error = loss_function(X @ beta, Y)
    return error

beta_init = np.array([1] * train_data.shape[1])
result = minimize(
    objective_function,
    beta_init,
    args=(train_data, train_label),
    method="BFGS",
    options={"maxiter": 500},
)
W = result.x
pred_label = [1 if i > 0 else -1 for i in (test_data.to_numpy() @ W)]
calc_accuracy_metric(pred_label, test_label)

# 2 - Logistic loss function
train_data.columns = train_data.columns.astype(str)
test_data.columns = test_data.columns.astype(str)

clf = LogisticRegression(random_state=0).fit(train_data, train_label)
pred_label = clf.predict(test_data)
calc_accuracy_metric(test_label, pred_label)

# 3 - Hinge loss function
est = svm.LinearSVC(random_state=0, max_iter=300, loss="hinge", tol=1e-8)
est.fit(train_data, train_label)
pred_label = est.predict(test_data)
calc_accuracy_metric(pred_label, test_label)

   0  1        2        3        4        5        6        7        8  \
0  1  0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1  1  0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2  1  0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3  1  0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4  1  0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   

         9  ...       25       26       27       28       29       30  \
0  0.03760  ... -0.51171  0.41078 -0.46168  0.21266 -0.34090  0.42267   
1 -0.04549  ... -0.26569 -0.20468 -0.18401 -0.19040 -0.11593 -0.16626   
2  0.01198  ... -0.40220  0.58984 -0.22145  0.43100 -0.17365  0.60436   
3  0.00000  ...  0.90695  0.51613  1.00000  1.00000 -0.20099  0.25682   
4 -0.16399  ... -0.65158  0.13290 -0.53206  0.02431 -0.62197 -0.05707   

        31       32       33  label  
0 -0.54487  0.18641 -0.45300      g  
1 -0.06288 -0.13738 -0.02447      b  
2 

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
